In [ ]:
pip install plotly==4.14.3

In [1]:
import requests
from bs4 import BeautifulSoup
import json
from datetime import date
from pymongo import MongoClient
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [56]:
client = MongoClient('mongo')
database = client['local']
collection = database['pixitag']
rankingDate = ''

In [72]:
main()

IndexError: list index out of range

In [61]:
def main():
    links = []
    day = int(date.today().strftime("%d")) - 1
    for i in range(1, day):
        if i < 10:
            stringDate = '&date=2021040' + str(i)
        else:
            stringDate = '&date=202104' + str(i)
        imageId = getLinks(stringDate, links)
        data = getData(imageId, links)
        ids = getExistingData()
        insertData(data, ids)
        print(i)

In [71]:
def getLinks(stringDate, links):
    url = "http://www.pixiv.net/ranking.php?mode=daily" + stringDate
    response = requests.get(url)
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36', 
              'password': 'rd.)T6vF',
              'pixiv_id' : 'flicfloclol@hotmail.fr'}
    response = requests.get(url, headers=headers, timeout = 10)
    parse = url.split('&')
    imageId = []


    if (len(parse) == 1):
        rankingDate = str(date.today())   
    else:
        rankingDate = parse[1][5:9] + '-' + parse[1][9:11] + '-' + parse[1][11:13]


    soup = BeautifulSoup(response.text)
    

    # for single illustration (different class)
    for link in soup.find_all('a', class_="work _work"):
        links.append("https://www.pixiv.net/ajax/illust/" + link.get('href').split('/')[2] + "?lang=en")
        imageId.append(link.get('href').split('/')[2])

    # for multiple illustrations (different class)\n",
    for link in soup.find_all('a', class_="work _work multiple"):
        links.append("https://www.pixiv.net/ajax/illust/" + link.get('href').split('/')[2] + "?lang=en")
        imageId.append(link.get('href').split('/')[2])

    return imageId


In [73]:
def getData(imageId, links):
    data = []

    counter = 0
    imageCounter = 0

    for url in links:

        response = requests.get(url, headers=headers, timeout = 100)
        jsonData = json.loads(response.text)

        for i in jsonData['body']['tags']['tags']:
            data.append({})
            if 'translation' not in i:
                if 'romaji' not in i:
                    data[counter].update({'tag': i['tag']})
                else:
                    data[counter].update({'tag': i['romaji']})
            else:
                data[counter].update({'tag': i['translation']['en']})

            data[counter].update({'date' : rankingDate})
            data[counter].update({'username' : jsonData['body']['userName']})
            data[counter].update({'imageId' : imageId[imageCounter]})
            counter += 1
        imageCounter += 1

    #print(data)
    return data



In [67]:
def getExistingData():
    cursor = collection.find({"date": rankingDate}, {"imageId", 'tag'})
    ids = []
    for document in cursor:
        ids.append(document)
    return ids

def insertData(data, ids):
    for i in data:
        if i:
            for dbData in ids:
                if dbData['imageId'] == i['imageId'] and dbData['tag'] == i['tag']:
                    i = {}
                    break
            if i:
                collection.insert_one(i)
                #print('Insert: ' + i['imageId'])


In [45]:
cursor = collection.find({'tag': 'original 3000+ bookmarks'})
ids = []
for document in cursor:
    ids.append(document)
    print(document)
    print('--------')

In [19]:
def resetCollection():
    collection.remove()
    print(collection)

In [8]:
#Get most popular tag by date
cursor = collection.aggregate([
    {"$match": {'date': '2021-04-17'}},
    {'$group': {
        '_id': '$tag',
        'nb': {'$sum': 1}
    }},
    {'$sort': {'nb': -1}}
])
for document in cursor:
    ids.append(document)
    print(document)

In [63]:
#Get images number per illustrator in ranking
cursor = collection.aggregate([
    #{"$match": {'date': '2021-04-17'}},
    {'$group': {
        '_id': {'username': '$username'},
        'images': {'$addToSet': '$imageId'}
    }},
    {'$project': {'nb': {'$size': '$images'}}},
    {'$sort': {'nb': -1}}
])
graphData = {'username': [], 'nb': []}
for document in cursor:
    #graphData.update({document['_id']['username']: [document['nb']]})
    graphData['username'].append(document['_id']['username'])
    graphData['nb'].append(document['nb'])
    #print(document)
#print(graphData)

In [64]:
#x = ['Product A', 'Product B', 'Product C']
#y = [20, 14, 23]

# Use the hovertext kw argument for hover text
fig = go.Figure(data=[go.Bar(x=graphData['username'], y=graphData['nb'])])
# Customize aspect
fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.6)
fig.update_layout(title_text='Image number per illustrator in ranking')
fig.show()

In [65]:
#Get 
cursor = collection.aggregate([
    {'$group': {
        '_id': {'tag': '$tag'},
        'date': {'$addToSet': '$date'}
    }},
    {'$project': {
        'nb': {'$size': '$date'}
    }},
    {'$match': {
        'nb': {'$gt': 3}
    }},
    {'$sort': {'nb': -1}}
])
graphData = {'tag': [], 'nb': []}
for document in cursor:
    graphData['tag'].append(document['_id']['tag'])
    graphData['nb'].append(document['nb'])
    print(document)

In [66]:
fig = go.Figure(data=[go.Bar(x=graphData['tag'], y=graphData['nb'])])
# Customize aspect
fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(0,0,0)',
                  marker_line_width=1.5, opacity=0.6)
fig.update_layout(title_text='Tag popularity by number of days')
fig.show()